In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, expr, when,\
desc, first, last, row_number, rank, substring
from pyspark.sql.functions import max as pmax, min as pmin
import pyspark as ps


In [3]:
import wget


In [4]:
fileName = 'https://people.bu.edu/kalathur/datasets/stocks_2013_2021.txt'
wget.download(fileName)


100% [............................................................................] 675159 / 675159

'stocks_2013_2021 (3).txt'

In [8]:
spark = SparkSession.builder.getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
stocksDF = spark.read.csv(
 "stocks_2013_2021.txt",
 header=False, inferSchema=True,
).toDF('Stock','Date','Open','High',
 'Low','Close','Volume'
 )
stocksDF.printSchema()
stocksDF.createOrReplaceTempView('stocks_table')


In [ ]:
#Problem 1 - DataFrames

In [ ]:
#Problem 1A
stocksDF.show(2)

In [ ]:
#Problem 1B 
stocksDF.count()

In [ ]:
#Problem 1C
stocksDF.groupBy("Stock").count().collect()

In [ ]:
#Problem 1 - SQL

In [ ]:
#Problem 1A
stocksSQL = spark.sql("SELECT * FROM STOCKS_TABLE LIMIT 2").show()


In [ ]:
#Problem 1
spark.sql("SELECT COUNT(*) FROM STOCKS_TABLE").show()

In [ ]:
#Problem 1C
spark.sql("SELECT STOCK, COUNT(*) FROM STOCKS_TABLE GROUP BY STOCK").show()

In [ ]:
#Problem 2 - DataFrames

In [ ]:
#Problem 2A
stocksDF.select(['Stock', 'Volume']).show(2)

In [ ]:
#Problem 2B
stockVolume = stocksDF.select(['Stock', 'Volume']).groupBy("Stock").sum("Volume").\
withColumnRenamed("sum(Volume)", "Total_Volume")
stockVolume.collect()

In [ ]:
#Problem 2C
stockVolume.sort("Total_Volume", ascending = False).collect()

In [ ]:
#Problem 2 - SQL

In [ ]:
#Problem 2A
spark.sql("SELECT STOCK, VOLUME FROM STOCKS_TABLE LIMIT 2").show()

In [ ]:
#Problem 2B
spark.sql("SELECT STOCK, SUM(VOLUME) TOTAL_VOLUME FROM STOCKS_TABLE GROUP BY STOCK").show()

In [ ]:
#Problem 2C
spark.sql("SELECT STOCK, SUM(VOLUME) TOTAL_VOLUME FROM STOCKS_TABLE GROUP BY STOCK ORDER BY TOTAL_VOLUME DESC").show()

In [ ]:
#Problem 3 - DataFrames

In [ ]:
#Problem 3A
stockOpenClose = stocksDF.select(["Stock", "Open", "Close"])
stockOpenClose.show(2)

In [ ]:
#Problem 3B
stockOpenClose.filter(stockOpenClose.Close > stockOpenClose.Open).groupBy("Stock").count().\
withColumnRenamed("count","Count")\
.sort("Count", ascending = False).show()

In [ ]:
#Problem 3 - SQL

In [ ]:
#Problem 3A
spark.sql("SELECT STOCK, OPEN, CLOSE FROM STOCKS_TABLE LIMIT 2").show()

In [ ]:
#Problem 3B
spark.sql("SELECT STOCK, SUM(CASE WHEN S.CLOSE > S.OPEN THEN 1 ELSE 0 END) TALLY FROM STOCKS_TABLE S GROUP BY STOCK \
          ORDER BY TALLY DESC").show()

In [ ]:
#Problem 4 - DataFrames

In [ ]:
#Problem 4A
highLow = stocksDF.select(["Stock", "High", "Low"])
highLow.show(2)

In [ ]:
#Problem 4B
highLow.groupBy("Stock").agg({"High": "max", "Low": "min"})\
.withColumnRenamed("max(High)", "Max_High").withColumnRenamed("min(low)", "Min_Low").show()

In [ ]:
#Problem 4 - SQL


In [ ]:
#Problem 4A
spark.sql("SELECT STOCK, HIGH, LOW FROM STOCKS_TABLE LIMIT 2").show()

In [ ]:
#Problem 4B
spark.sql("SELECT STOCK, MAX(HIGH) MAX_HIGH, MIN(LOW) MAX_LOW FROM STOCKS_TABLE GROUP BY STOCK").show()

In [ ]:
#Problem 5 - DataFrames

In [ ]:
#Problem 5A
highLow = stocksDF.select(col("Stock"), col("High"), col("Date").alias("HighDate"), \
                          col("Low"), col("Date").alias("LowDate"))
highLow.show(5)


In [ ]:
#Problem 5B


In [ ]:
#Problem 5 - SQL

In [ ]:
#Problem 5A
spark.sql("SELECT STOCK, HIGH, DATE HIGHDATE, LOW, DATE LOWDATE FROM STOCKS_TABLE LIMIT 5").show()


In [ ]:
#Problem 5B
spark.sql("SELECT STOCK, HIGH, DATE FROM STOCKS_TABLE A JOIN \
        (SELECT STOCK TEMPTICK, MAX(HIGH) MAXHIGH FROM STOCKS_TABLE GROUP BY STOCK) B ON \
        A.STOCK = B.TEMPTICK AND A.HIGH = B.MAXHIGH \
        UNION \
          SELECT STOCK, LOW, DATE FROM STOCKS_TABLE A JOIN \
        (SELECT STOCK TEMPTICK, MIN(LOW) MINLOW FROM STOCKS_TABLE GROUP BY STOCK) B ON \
        A.STOCK = B.TEMPTICK AND A.LOW = B.MINLOW ORDER BY STOCK").show()

In [ ]:
#Problem 6 - DataFrames

In [ ]:
stockYearVolume = stocksDF.select(["Stock", "Date", "Volume"]).show(5)


In [ ]:
#Problem 6A
stockYearVolume = stocksDF.select(["Stock", "Date", "Volume"])
stockYearVolume = stockYearVolume.select(col("Stock"), substring(col("Date"), 0, 4), col("Volume"))\
.withColumnRenamed("substring(Date, 0, 4)", "Year")
stockYearVolume.show(5)


In [ ]:
#Problem 6 - SQL

In [ ]:
#Problem 6A
spark.sql("SELECT STOCK, LEFT(DATE, 4) YEAR, VOLUME FROM STOCKS_TABLE").show(5)

In [ ]:
#Problem 6B
spark.sql("SELECT STOCK, YEAR, SUM(VOLUME) FROM (SELECT STOCK, LEFT(DATE, 4) YEAR, VOLUME FROM STOCKS_TABLE) B \
GROUP BY STOCK, YEAR ORDER BY STOCK, YEAR").show()
      

In [ ]:
#Problem 7 - Data Frames

In [ ]:
#Problem 7 - SQL

In [ ]:
#Problem 7A
spark.sql("SELECT STOCK, LEFT(DATE, 4) YEAR, HIGH, LOW FROM STOCKS_TABLE").show(2)

In [ ]:
#Problem 7B
spark.sql("SELECT STOCK, LEFT(DATE, 4) YEAR, MAX(HIGH) MAX_HIGH, MIN(LOW) MIN_LOW FROM STOCKS_TABLE \
GROUP BY STOCK, YEAR ORDER BY STOCK, YEAR").show()

In [ ]:
#Problem 7C
spark.sql("SELECT A.STOCK, B.YEAR, A.HIGH, A.DATE, A.LOW, A.DATE FROM STOCKS_TABLE A JOIN \
          (SELECT STOCK TEMPTICKER, LEFT(DATE, 4) YEAR, MAX(HIGH) MAX_HIGH, MIN(LOW) MIN_LOW FROM STOCKS_TABLE \
GROUP BY STOCK, YEAR) B \
ON A.STOCK = B.TEMPTICKER AND ")